In [1]:
import pandas as pd 
import numpy as np 
%matplotlib inline 
import matplotlib.pyplot as plt 
from os import listdir
from keras.preprocessing import sequence 
import tensorflow as tf 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import LSTM 
from keras.optimizers import Adam 
from keras.models import load_model 
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [6]:
df1 = pd.read_csv('dataset/MovementAAL_RSS_1.csv') 
df2 = pd.read_csv('dataset/MovementAAL_RSS_2.csv')
df1.head()

,#RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4
0,-0.90476,-0.48,0.28571,0.30
1,-0.57143,-0.32,0.14286,0.30
2,-0.38095,-0.28,-0.14286,0.35
3,-0.28571,-0.20,-0.47619,0.35
4,-0.14286,-0.20,0.14286,-0.20


In [7]:
df2.head()

,#RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4
0,-0.57143,-0.20,0.71429,0.50
1,-0.76190,-0.48,0.76190,-0.25
2,-0.85714,-0.60,0.85714,0.55
3,-0.76190,-0.40,0.71429,0.60
4,-0.76190,-0.84,0.85714,0.45


In [8]:
df1.shape, df2.shape

((27, 4), (26, 4))

In [9]:
path = 'dataset/MovementAAL_RSS_'
sequences = list()
for i in range(1,315):
    file_path = path + str(i) + '.csv'
    print(file_path)
    df = pd.read_csv(file_path, header=0)
    values = df.values
    sequences.append(values)
targets = pd.read_csv('dataset/MovementAAL_target.csv') 
targets = targets.values[:,1]

dataset/MovementAAL_RSS_1.csv
dataset/MovementAAL_RSS_2.csv
dataset/MovementAAL_RSS_3.csv
dataset/MovementAAL_RSS_4.csv
dataset/MovementAAL_RSS_5.csv
dataset/MovementAAL_RSS_6.csv
dataset/MovementAAL_RSS_7.csv
dataset/MovementAAL_RSS_8.csv
dataset/MovementAAL_RSS_9.csv
dataset/MovementAAL_RSS_10.csv
dataset/MovementAAL_RSS_11.csv
dataset/MovementAAL_RSS_12.csv
dataset/MovementAAL_RSS_13.csv
dataset/MovementAAL_RSS_14.csv
dataset/MovementAAL_RSS_15.csv
dataset/MovementAAL_RSS_16.csv
dataset/MovementAAL_RSS_17.csv
dataset/MovementAAL_RSS_18.csv
dataset/MovementAAL_RSS_19.csv
dataset/MovementAAL_RSS_20.csv
dataset/MovementAAL_RSS_21.csv
dataset/MovementAAL_RSS_22.csv
dataset/MovementAAL_RSS_23.csv
dataset/MovementAAL_RSS_24.csv
dataset/MovementAAL_RSS_25.csv
dataset/MovementAAL_RSS_26.csv
dataset/MovementAAL_RSS_27.csv
dataset/MovementAAL_RSS_28.csv
dataset/MovementAAL_RSS_29.csv
dataset/MovementAAL_RSS_30.csv
dataset/MovementAAL_RSS_31.csv
dataset/MovementAAL_RSS_32.csv
dataset/MovementA

In [10]:
sequences[0]

array([[-0.90476 , -0.48    ,  0.28571 ,  0.3     ],
       [-0.57143 , -0.32    ,  0.14286 ,  0.3     ],
       [-0.38095 , -0.28    , -0.14286 ,  0.35    ],
       [-0.28571 , -0.2     , -0.47619 ,  0.35    ],
       [-0.14286 , -0.2     ,  0.14286 , -0.2     ],
       [-0.14286 , -0.2     ,  0.047619,  0.      ],
       [-0.14286 , -0.16    , -0.38095 ,  0.2     ],
       [-0.14286 , -0.04    , -0.61905 , -0.2     ],
       [-0.095238, -0.08    ,  0.14286 , -0.55    ],
       [-0.047619,  0.04    , -0.095238,  0.05    ],
       [-0.19048 , -0.04    ,  0.095238,  0.4     ],
       [-0.095238, -0.04    , -0.14286 ,  0.35    ],
       [-0.33333 , -0.08    , -0.28571 , -0.2     ],
       [-0.2381  ,  0.04    ,  0.14286 ,  0.35    ],
       [ 0.      ,  0.08    ,  0.14286 ,  0.05    ],
       [-0.095238,  0.04    ,  0.095238,  0.1     ],
       [-0.14286 , -0.2     ,  0.14286 ,  0.5     ],
       [-0.19048 ,  0.04    , -0.42857 ,  0.3     ],
       [-0.14286 , -0.08    , -0.2381  ,  0.15

In [11]:
groups = pd.read_csv('groups/MovementAAL_DatasetGroup.csv', header=0) 
groups = groups.values[:,1]

In [12]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
pd.Series(len_sequences).describe()

count    314.000000
mean      42.028662
std       16.185303
min       19.000000
25%       26.000000
50%       41.000000
75%       56.000000
max      129.000000
dtype: float64

In [13]:
#Padding the sequence with the values in last row to max length
to_pad = 129
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(4, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

#truncate the sequence to length 60
from keras.preprocessing import sequence
seq_len = 60
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [14]:
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==2)]
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==1]
test = [final_seq[i] for i in range(len(groups)) if groups[i]==3]

train_target = [targets[i] for i in range(len(groups)) if (groups[i]==2)]
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==1]
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

train_target = np.array(train_target)
train_target = (train_target+1)/2

validation_target = np.array(validation_target)
validation_target = (validation_target+1)/2

test_target = np.array(test_target)
test_target = (test_target+1)/2

In [15]:
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 4)))
model.add(Dense(1, activation='sigmoid'))
model.summary()

W0924 09:42:39.981674 24092 deprecation_wrapper.py:119] From C:\Users\igaraybeng001\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0924 09:42:40.011594 24092 deprecation_wrapper.py:119] From C:\Users\igaraybeng001\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0924 09:42:40.015581 24092 deprecation_wrapper.py:119] From C:\Users\igaraybeng001\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               267264    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 267,521
Trainable params: 267,521
Non-trainable params: 0
_________________________________________________________________


In [16]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=128, callbacks=[chk], validation_data=(validation,validation_target))
#loading the model and checking accuracy on the test data
model = load_model('best_model.pkl')

from sklearn.metrics import accuracy_score
test_preds = model.predict_classes(test)
accuracy_score(test_target, test_preds)

W0924 09:42:52.166744 24092 deprecation_wrapper.py:119] From C:\Users\igaraybeng001\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0924 09:42:52.180706 24092 deprecation_wrapper.py:119] From C:\Users\igaraybeng001\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0924 09:42:52.296395 24092 deprecation.py:323] From C:\Users\igaraybeng001\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0924 09:42:55.821949 24092 deprecation_wrapper.py:119] From C:\Users\igaraybeng001\AppData\Local\Continuum\anac

Train on 106 samples, validate on 104 samples
Epoch 1/200
106/106 [==============================] - 5s 48ms/step - loss: 0.6941 - acc: 0.4528 - val_loss: 0.6821 - val_acc: 0.5865

Epoch 00001: val_acc improved from -inf to 0.58654, saving model to best_model.pkl
Epoch 2/200
106/106 [==============================] - 1s 5ms/step - loss: 0.6828 - acc: 0.5566 - val_loss: 0.6726 - val_acc: 0.5865

Epoch 00002: val_acc did not improve from 0.58654
Epoch 3/200
106/106 [==============================] - 1s 9ms/step - loss: 0.6727 - acc: 0.5849 - val_loss: 0.6647 - val_acc: 0.5673

Epoch 00003: val_acc did not improve from 0.58654
Epoch 4/200
106/106 [==============================] - 1s 6ms/step - loss: 0.6632 - acc: 0.6038 - val_loss: 0.6566 - val_acc: 0.5192

Epoch 00004: val_acc did not improve from 0.58654
Epoch 5/200
106/106 [==============================] - 1s 8ms/step - loss: 0.6521 - acc: 0.6415 - val_loss: 0.6482 - val_acc: 0.5192

Epoch 00005: val_acc did not improve from 0.58654


0.8173076923076923